# Snowpark Basics HoL Part 1 - DataFrame Basics

## 1.1 Setup

### Imports
These imports are from our local Python environment, snowparkbasics. Look out for F. and T. below.

In [ ]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

import sys
import json
import pandas as pd
import numpy as np

# Make sure we do not get line breaks when doing show on wide dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Create Snowpark Session
Using a credentials file simplifies the HoL but is not recommended as good practice for development or production environments.
<br> The Python connector documentation explains how to use other authentication methods.

In [ ]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [ ]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

### Modifying our Session
We can use **session.sql** to issue any 'SQL' command. Note that due to lazy evaluation, typically nothing will happen without a show() or collect().

In [ ]:
session.sql("USE WAREHOUSE TASTY_DEV_WH").collect()
print(f"Current Warehouse: {session.get_current_warehouse()}")

However, session also has a number of methods such as **use_warehouse()**. These *are* run immediately.

In [ ]:
session.use_warehouse("TASTY_DE_WH")
print(f"Current Warehouse: {session.get_current_warehouse()}")

## 1.2 Loading a DataFrame

### Pandas DataFrames from CSV
Let's create a Pandas dataframe directly from csv.

In [ ]:
# Creating a Pandas DataFrame - the order header csv is in fact the data from only one truck!!
pandas_truck_df = pd.read_csv('data/truck.csv')
pandas_header_df = pd.read_csv('data/header.csv')
print(type(pandas_truck_df))

In [ ]:
# Displaying the Pandas dataframe
pandas_truck_df

### Snowpark DataFrames from Tables

The Snowpark **Table** class is a child of the **DataFrame** class.  We can define a dataframe based on a table very simply.
<br> (We'll look at loading file data into tables in Part 3.)

In [ ]:
snowpark_truck_df = session.table('TRUCK')
snowpark_header_df = session.table('ORDER_HEADER')
print(type(snowpark_truck_df))

### Comparing DataFrames
Compare sizes

In [ ]:
print(
    "Size in MB of Pandas Truck DataFrame in Memory: ",
    np.round(sys.getsizeof(pandas_truck_df) / (1024.0**2), 2),
)
print(
    "Size in MB of Snowpark Truck DataFrame in Memory: ",
    np.round(sys.getsizeof(snowpark_truck_df) / (1024.0**2), 2),
)
print(
    "Size in MB of Pandas Header DataFrame in Memory: ",
    np.round(sys.getsizeof(pandas_header_df) / (1024.0**2), 2),
)
print(
    "Size in MB of Snowpark Header DataFrame in Memory: ",
    np.round(sys.getsizeof(snowpark_header_df) / (1024.0**2), 2),
)

The only thing stored in a Snowpark DataFrame is the SQL needed to return data.
<br>Trying to manipulate even one truck's worth of order headers in Pandas starts to get 'interesting'.

Now, what is going on under the covers? You might want to log into your Snowflake account as the same user and review Snowsight Query History. But you can also use this DataFrame attribute from Snowpark...

In [ ]:
snowpark_header_df.queries

A Snowpark DataFrame can be converted to a Pandas DataFrame. This will pull the data from Snowflake into local memory.

In [ ]:
pandas_truck_df2 = snowpark_truck_df.to_pandas()
pandas_truck_df2

Both our Pandas DataFrames have the same shape

In [ ]:
pandas_truck_df.shape, pandas_truck_df2.shape

### Displaying a Snowpark DataFrame
Defining and modifying a Snowpark dataframe does not generally result in any activity within Snowflake - lazy evaluation, similar to Spark.
The **show** method causes a query to be generated and sent and data returned - by default just 10 rows.
In contrast **toPandas** or **to_pandas** will retrieve the whole dataset unless you set a **limit**. 

In [ ]:
snowpark_header_df.show() # <- has a default limit of 10, and prints the data out
snowpark_header_df.limit(5).toPandas() # <- collects first 5 rows and displays as pandas-dataframe

In fact, you don't need to give your dataframe a name just to examine the table.

In [ ]:
session.table("RAW_POS.LOCATION").show()

### Simple DataFrame Information

The **count** method on a DataFrame will return the number of rows. This also triggers a query to Snowflake. *(Cf pyspark.sql.DataFrame.count())*

In [ ]:
# Number of rows in dataset
snowpark_header_df.count()

We can get an idea of the structure from the **schema** attribute.  *(Cf pyspark.sql.DataFrame.schema)*

In [ ]:
header_schema = snowpark_header_df.schema
header_schema

Using the **describe** method will return some basic statistics for all numeric and string columns.  *(Cf pyspark.sql.DataFrame.describe())*
<br>Note that this does real work inside Snowflake! The statistical values are not necessarily meaningful for all columns.
<br>Can you find the maxiumum order value in the data? 

In [ ]:
# Calculating various statistics per column
snowpark_header_df.describe().show()

## 1.3 Managing Columns

### Selecting Columns
There are several ways to **select** specific columns, including **functions.col** and **DataFrame.col**. 
<br>The latter two are needed in several stuations to avoid ambiguities with string constants. 
<br>What do you notice about the four results below?

In [ ]:
header_df1 = snowpark_header_df.select('ORDER_ID','TRUCK_ID','LOCATION_ID','ORDER_AMOUNT','ORDER_TS')
header_df2 = snowpark_header_df[['ORDER_ID','TRUCK_ID','LOCATION_ID','ORDER_AMOUNT','ORDER_TS']] # -> pandas-like selection
header_df3 = snowpark_header_df.select(F.col("ORDER_ID"),F.col("truck_id"),F.col("location_id"),
                                       F.col("order_amount"), F.col("order_ts"))
header_df4 = snowpark_header_df.select(snowpark_header_df.col('ORDER_ID'),snowpark_header_df.col('TRUCK_ID'),
                                       snowpark_header_df.col('LOCATION_ID'),snowpark_header_df.col('ORDER_AMOUNT'),
                                       snowpark_header_df.col('ORDER_TS'))
header_df1.show()
header_df2.show()
header_df3.show()
header_df4.show()

In general in Python single and double quotes are interchangeable. Best practice is to choose one and stick with it (unlike in this HoL). 
<br>Note that in all the examples above, the names are implicitly converted to uppercase.
<br>To handle identifiers with lowercase you need to add explicit double quotes within the string, either within single quotes as below, or with an escape character. 

In [ ]:
#The following statement should fail
snowpark_header_df.select(F.col('"order_id"'),F.col("TRUCK_ID"),F.col("LOCATION_ID"),F.col("ORDER_AMOUNT"), F.col("ORDER_TS")).show()

### Casting, Aliasing and In-Line Calculations
We can **cast** the column datatypes. For example ORDER_AMOUNT could be cast to NUMBER(36,2). Alternatively we have functions like **to_date**.

In [ ]:
header_df1 = snowpark_header_df.select(F.col("ORDER_ID"),F.col('ORDER_AMOUNT').cast(T.DecimalType(36,2)),F.to_date(F.col('ORDER_TS')))
header_df1.show()

That's a bit ugly. Let's alias those columns...  **alias**, **name** and **as_** all achieve the same effect. *(Cf pyspark alias or name)*

In [ ]:
header_df1 = snowpark_header_df.select(F.col("ORDER_ID"),F.col('ORDER_AMOUNT').cast(T.DecimalType(36,2)).alias("ORDER_AMOUNT_2D"),
                                      F.to_date(F.col('ORDER_TS')).alias('ORDER_DATE'))
header_df1.show()
header_df1.queries

We can also include calculated expressions within a select as we can in SQL. For example we can use + - * / ** arithmetic operators.

In [ ]:
header_df2 = header_df1.select(F.col("ORDER_ID"),F.col('ORDER_AMOUNT_2D'),
                              (F.col('ORDER_AMOUNT_2D')*100).alias("OA_CENTS"))
header_df2.show()

Again, switch to Snowsight Query History and see what is going on...

### Adding and Removing Columns


To add a new calculated column to a Snowpark DataFrame the **withColumn** or **with_column** method can be used.  *(Cf pysaprk withColumn)*
In this example we are adding a new TRUCK column, AGE, that calculates the number of years since the YEAR. 
Note the use of F.col here - otherwise 'YEAR' could be seen as a string value. One approach is to use built-in Python functions to derive the current year locally.

In [ ]:
import datetime
from datetime import datetime
year = datetime.now().year

truck_df1 = snowpark_truck_df.select('TRUCK_ID','REGION','ISO_COUNTRY_CODE','YEAR',
                                     'MAKE','MODEL','TRUCK_OPENING_DATE')
truck_df1 = truck_df1.withColumn('AGE', year - F.col('YEAR'))
truck_df1.to_pandas()

The following version pushes the current year 'calculation' down to Snowflake.

In this section we show how each new version of the dataframe can replace the previous one by using the same name.
This can make sense whilst we build up the dataframe query we really want. However, when we do this across cells and try to rerun just one cell we can get errors if a later statement has altered the structure that an earlier statement relied on.... We avoid that here by redefining truck_df1 from its source.

In [ ]:
truck_df1 = snowpark_truck_df.select('TRUCK_ID','REGION','ISO_COUNTRY_CODE','YEAR',
                                     'MAKE','MODEL','TRUCK_OPENING_DATE')
truck_df1 = truck_df1.withColumn('AGE', F.date_part("year", F.current_date()) - F.col('YEAR'))
truck_df1.to_pandas()

If we do not want to use specific columns we can use **drop** to remove those from a Snowpark DataFrame.  
**Note:** This is not removing them from the underlying table.

In [ ]:
# Drop a column
truck_df1 = truck_df1.drop('MODEL','YEAR')
truck_df1.show()

## 1.4 Simple Data Manipulation

### Filtering Rows
To filter/select specific rows we use **filter**.
A whole set of column operators are available to be used e.g. 

==, !=, <, <=, >, >=  for comparisons;   &, |  and or;  + - * / **  arithmetic operators

In [ ]:
# Filter data
truck_df2 = truck_df1.filter(F.col('ISO_COUNTRY_CODE') == 'GB')
truck_df2.show()
truck_df3 = truck_df1.filter(F.col('ISO_COUNTRY_CODE').in_('ES','FR','GB')).sort('ISO_COUNTRY_CODE')
truck_df3.show()
truck_df4 = truck_df1.filter(F.col('ISO_COUNTRY_CODE').like('F%'))
truck_df4.show()
truck_df3.queries

### Sorting
We may want to see data in a specific order. For this the **sort** method is used...

In [ ]:
# Sort data
truck_df4 = truck_df4.sort(F.col('TRUCK_OPENING_DATE').desc(),F.col('ISO_COUNTRY_CODE'))
truck_df4.show(20)

### Aggregation
To aggregate data the **groupBy** or **group_by** method is typically used. The groupby method produces a RelationalGroupedDataFrame object with its own specific methods, which, in turn, return a DataFrame. The **agg** method provides the most flexibility for managing the output, and including different aggregate metrics for different columns. Note the syntax - although operating on columns, the functions like avg expect the string of the column name.

In [ ]:
truck_df5 = truck_df3.groupBy(['ISO_COUNTRY_CODE','MAKE']).agg(
             [F.count('*').alias('COUNT'),F.avg('AGE').alias('AVG_TRUCK_AGE'),F.max('TRUCK_ID').alias('MAX_TRUCK_ID')])
truck_df5 = truck_df5.sort(F.col('ISO_COUNTRY_CODE'), F.col('COUNT').desc())
truck_df5.queries

In [ ]:
truck_df5.show(15)

### Using SQL
How might we express the same combined query in SQL? It is quite likely that we would want to break it down in a similar way.
We can run SQL queries directly using **session.sql** (including Snowflake commands issued as SQL). 
Note that nothing will happen without a collect() or show().

In the example below, the three quotes beginning and end are how we indicate a multi-line string in Python.

In [ ]:
truck_df6 = session.sql("""
 SELECT ISO_COUNTRY_CODE, MAKE, count(1) AS COUNT, avg(AGE) AS AVG_TRUCK_AGE, max(TRUCK_ID) AS MAX_TRUCK_ID 
   FROM ( SELECT TRUCK_ID, REGION, ISO_COUNTRY_CODE, MAKE, TRUCK_OPENING_DATE, (date_part('year', current_date()) - YEAR) AS AGE 
       FROM TRUCK WHERE ISO_COUNTRY_CODE IN ('ES', 'FR', 'GB')
     ) 
   GROUP BY ISO_COUNTRY_CODE, MAKE ORDER BY ISO_COUNTRY_CODE ASC NULLS FIRST, COUNT DESC NULLS LAST LIMIT 15
   """)
truck_df6.show()
truck_df6.queries

## 1.5 Persist Transformations

If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.  
**save_as_table** saves the result in a table, if **mode='overwrite'** then it will also replace the data that is in it.

In [ ]:
truck_df3.write.save_as_table(table_name='TRUCK_ANALYSIS', mode='overwrite')
session.table('TRUCK_ANALYSIS').show()

## 1.X YOUR TURN!

Here is the challenge: Generate a list of months for which we have data, the total order amount for each month (assume amounts are all held in the same currency), and the number of distinct locations visted in each month.
<br>Hints:
Functions you may find useful include **count_distinct** (aka countDistinct), **date_part**, **to_char** with numeric formatting '09' or 'FM09' and **concat**.

### Hint:   
To see all methods available use the TAB key.    F.<TAB>   Will show all functon methods.

To see for a specific function the help text.   SHIFT-TAB
                                                                                                                                                      
https://docs.snowflake.com/developer-guide/snowpark/reference/scala/com/snowflake/snowpark/index.html

### Check out the data

What columns in Order Header will you need?  

In [ ]:
exercise_header_df = session.table('ORDER_HEADER')
exercise_header_df.show()

### Select the columns you need and create a Month column
You'll need to concatenate the year and month date parts of the order timestamp.
Note that (currently) date_format in Snowpark is an alias of to_date and not the date to string functionality of Pyspark...

In [ ]:
header_df1 = exercise_header_df.select(F.col('ORDER_ID'), F.col('LOCATION_ID'), 
        F.col('ORDER_AMOUNT').cast(T.DecimalType(36,2)).alias("ORDER_AMOUNT"),
        F.concat(F.to_char(F.date_part("year",'ORDER_TS')), F.to_char(F.date_part("month",'ORDER_TS'),'FM09')).alias('ORDER_MONTH'))
header_df1.show()

In [ ]:
header_df2 = exercise_header_df.select(F.col('ORDER_ID'), F.col('LOCATION_ID'), 
        F.col('ORDER_AMOUNT').cast(T.DecimalType(36,2)).alias("ORDER_AMOUNT"),
        F.date_trunc("MONTH", F.col('ORDER_TS')).alias('ORDER_MONTH'))
header_df2.show()

### Now aggregate by month


In [ ]:
header_groupdf = header_df1.group_by('ORDER_MONTH').agg([F.count_distinct('LOCATION_ID').alias('LOCATIONS'),
                                                         F.sum('ORDER_AMOUNT').alias('TOTAL_ORDER_AMOUNT')]).sort('ORDER_MONTH')
header_groupdf.limit(20).toPandas()

In [ ]:
session.close()